In [18]:
import searoute as sr
from copy import copy

from searoute.classes.passages import Passage
import geojson

In [19]:
# Function to find edges with a specific attribute value
def find_edges_by_attribute(G, passage_values):
    passage_list = {key: [] for key in passage_values}

    for u, v, attr in G.edges(data=True):
        for attr_value in passage_values:
            if attr.get('passage') == attr_value:
                passage_list[attr_value].append((u, v))
    return passage_list


def get_middle_element(lst):
    length = len(lst)
    middle_index = length // 2  # Integer division
    
    if length % 2 == 0:
        # Even length: return the element just before the middle
        return lst[middle_index]
    else:
        # Odd length: return the middle element
        return lst[middle_index]
    
def get_features_with_stops( origin, dest, mandatory_passages):
    M:sr.Marnet = copy(sr.setup_M())

    passage_pts = find_edges_by_attribute(M, mandatory_passages)
    lst = []
    for key, value in passage_pts.items():
        mid_pt = get_middle_element(value)
        # get second point of found passage
        yp,xp = mid_pt[1]
        lst.append(((yp,xp), key))

    fts = []
    step = origin
    #add last position
    lst.append((dest, None))
    for stop in lst:
        stop_i = stop[0]
        traj = sr.searoute(step, stop_i, append_orig_dest=True, return_passages=True)
        fts.append(traj)
        step = stop_i

    return fts

def get_features_with_stops( origin, dest, mandatory_passages):
    M:sr.Marnet = copy(sr.setup_M())

    passage_pts = find_edges_by_attribute(M, mandatory_passages)
    lst = []
    for key, value in passage_pts.items():
        mid_pt = get_middle_element(value)
        # get second point of found passage
        yp,xp = mid_pt[1]
        lst.append(((yp,xp), key))

    fts = []
    step = origin
    #add last position
    lst.append((dest, None))
    for stop in lst:
        stop_i = stop[0]
        traj = sr.searoute(step, stop_i, append_orig_dest=True, return_passages=True)
        fts.append(traj)
        step = stop_i

    return fts

def append_features(features):
    """
    Appends a list of GeoJSON features into a single FeatureCollection.

    Parameters:
    - features (List[geojson.Feature]): A list of GeoJSON features to be appended.

    Returns:
    - geojson.FeatureCollection: A GeoJSON FeatureCollection containing all features.
    """
    # Create a FeatureCollection with the list of features
    feature_collection = geojson.FeatureCollection(features)
    
    return feature_collection

In [20]:
origin = [-61.87, 17.15]# barbuda  
dest =   [52.99, 25.01] # dubai 
# barbuda to dubai 
mandatory_passages = [ Passage.panama,Passage.chili, Passage.south_africa, Passage.gibraltar, Passage.suez] 
# the way around dubai to barbuda 
#mandatory_passages = [  Passage.suez,  Passage.gibraltar, Passage.south_africa, Passage.chili,Passage.panama] 


fts = get_features_with_stops(origin, dest, mandatory_passages) 
print(append_features(fts)) 

traversed_passages = [] 
for ft in fts: 
    passed = ft['properties']['traversed_passages'] 
    traversed_passages.extend(passed) 

traversed_passages = set(traversed_passages) 

assert all(element in traversed_passages for element in mandatory_passages) 

{"features": [{"geometry": {"coordinates": [[-61.87, 17.15], [-62.0493, 17.2795], [-62.25, 17], [-62.7359, 17.0444], [-64.177577, 17.153263], [-64.9859, 17.2143], [-65.827306, 17.258974], [-66.274325, 17.05734], [-67.078784, 16.694479], [-68.6191, 15.9997], [-69.392684, 15.630755], [-69.848385, 15.413417], [-70.568459, 15.069992], [-70.79342, 14.962702], [-71.738548, 14.511942], [-72.4379, 14.1784], [-73.777685, 13.507848], [-76.1956, 12.2977], [-80, 9.75], [-79.900612, 9.208176], [-79.821304, 9.173012], [-79.803246, 9.118307], [-79.741379, 9.118236], [-79.689331, 9.10639]], "type": "LineString"}, "properties": {"duration_hours": 51.264329203324586, "length": 2278.596904429371, "traversed_passages": ["panama"], "units": "km"}, "type": "Feature"}, {"geometry": {"coordinates": [[-79.689331, 9.10639], [-79.641747, 9.035982], [-79.585922, 8.992656], [-79.494118, 8.796576], [-79.5, 8.6], [-80, 7], [-80.412834, 5.194964], [-80.9033, 3.0505], [-81.176151, 1.84843], [-81.8, -0.9], [-81.25, -3]